In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products.csv'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [2]:
import re
import string
from nltk.corpus import stopwords
from bs4 import BeautifulSoup
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
df = pd.read_csv('Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products.csv')

In [4]:
df.head()

,id,name,primaryCategories,reviews.doRecommend,reviews.rating,reviews.text
0,AVqVGZNvQMlgsOJE6eUY,"Amazon Kindle E-Reader 6"" Wifi (8th Generation...",Electronics,False,3,I thought it would be as big as small paper bu...
1,AVqVGZNvQMlgsOJE6eUY,"Amazon Kindle E-Reader 6"" Wifi (8th Generation...",Electronics,True,5,This kindle is light and easy to use especiall...
2,AVqVGZNvQMlgsOJE6eUY,"Amazon Kindle E-Reader 6"" Wifi (8th Generation...",Electronics,False,4,Didnt know how much i'd use a kindle so went f...
3,AVqVGZNvQMlgsOJE6eUY,"Amazon Kindle E-Reader 6"" Wifi (8th Generation...",Electronics,True,5,I am 100 happy with my purchase. I caught it o...
4,AVqVGZNvQMlgsOJE6eUY,"Amazon Kindle E-Reader 6"" Wifi (8th Generation...",Electronics,True,5,Solid entry level Kindle. Great for kids. Gift...


In [5]:
df = df.drop(["id", "name","primaryCategories","reviews.doRecommend"], axis=1)

In [6]:
df = df.sample(frac=1)

In [7]:
df.head()

,reviews.rating,reviews.text
1557,4,Good product for a young child to play with wi...
4811,5,Only negative is the apps. Thought they were f...
3412,4,Good for reading white and blackScreen is stil...
3667,5,"I have the HD 7 and an HDX 7, and the 8 outper..."
4842,3,Decent little tablet if you are not a game pla...


In [8]:
from sklearn import preprocessing 

# label_encoder object knows how to understand word labels. 
label_encoder = preprocessing.LabelEncoder() 

# Encode labels in column 'species'. 
df['reviews.rating']= label_encoder.fit_transform(df['reviews.rating']) 

df['reviews.rating'].unique() 

array([3, 4, 2, 1, 0], dtype=int64)

In [9]:
df.head()

,reviews.rating,reviews.text
1557,3,Good product for a young child to play with wi...
4811,4,Only negative is the apps. Thought they were f...
3412,3,Good for reading white and blackScreen is stil...
3667,4,"I have the HD 7 and an HDX 7, and the 8 outper..."
4842,2,Decent little tablet if you are not a game pla...


In [10]:
#dataset description #truthful=1 deceptive=0
df.groupby('reviews.rating').describe()

reviews.text         \
                      count unique   
reviews.rating                       
0                        63     48   
1                        54     47   
2                       197    175   
3                      1208   1055   
4                      3478   3061   

                                                                        
                                                              top freq  
reviews.rating                                                          
0               Bought this mostly as a backup.and to read a f...    4  
1               Not a very responsive program. Very limited in...    2  
2               bought a few kindles in the past but this time...    3  
3               A little more complicated than my original kin...    4  
4               This was a Christmas present. The person that ...    4

In [11]:
def clean_text(text):
    
    ## Remove puncuation
    text = text.translate(string.punctuation)
    
    ## Convert words to lower case and split them
    text = text.lower().split()
    
    ## Remove stop words
    stops = set(stopwords.words("english"))
    text = [w for w in text if not w in stops and len(w) >= 3]
    
    text = " ".join(text)

    # Clean the text
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)
    
    text = text.split()
    stemmer = SnowballStemmer('english')
    stemmed_words = [stemmer.stem(word) for word in text]
    text = " ".join(stemmed_words)

    return text

In [12]:
# Some preprocesssing that will be common to all the text classification methods

puncts = [',', '.', '"', ':', ')', '(', '-', '!', '?', '|', ';', "'", '$', '&', '/', '[', ']', '>', '%', '=', '#', '*', '+', '\\', '•',  '~', '@', '£', 
 '·', '_', '{', '}', '©', '^', '®', '`',  '<', '→', '°', '€', '™', '›',  '♥', '←', '×', '§', '″', '′', 'Â', '█', '½', 'à', '…', 
 '“', '★', '”', '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶', '↑', '±', '¿', '▾', '═', '¦', '║', '―', '¥', '▓', '—', '‹', '─', 
 '▒', '：', '¼', '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲', 'è', '¸', '¾', 'Ã', '⋅', '‘', '∞', 
 '∙', '）', '↓', '、', '│', '（', '»', '，', '♪', '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 'ï', 'Ø', '¹', '≤', '‡', '√', ]

def clean_char(x):
    x = str(x)
    for punct in puncts:
        if punct in x:
            x = x.replace(punct, f' {punct} ')
    return x

In [13]:
def clean_numbers(x):
    if bool(re.search(r'\d', x)):
        x = re.sub('[0-9]{5,}', '#####', x)
        x = re.sub('[0-9]{4}', '####', x)
        x = re.sub('[0-9]{3}', '###', x)
        x = re.sub('[0-9]{2}', '##', x)
    return x

In [14]:
df['reviews.text'] = df['reviews.text'].map(lambda a: clean_numbers(a))

In [15]:
df['reviews.text'] = df['reviews.text'].map(lambda a: clean_char(a))

In [16]:
df['reviews.text'] = df['reviews.text'].map(lambda a: clean_text(a))

In [17]:
df['reviews.text']

1557          good product young child play without break
4811       negat app thought android amazon version limit
3412              good read white blackscreen still small
3667    hdx outperform amazon fan eco system must scre...
4842    decent littl tablet game player tablet shut ca...
                              ...                        
3383    new design lighter longer batteri life page di...
4559                      good tablet babi keep busi love
3871    price guy cannot beat great watch video stream...
1620    great tablet year old kid friend safe mom dad ...
267     amazon echo show great help daili rutin ask si...
Name: reviews.text, Length: 5000, dtype: object

In [18]:
df.describe()

,reviews.rating
count,5000.000000
mean,3.596800
std,0.731804
min,0.000000
25%,3.000000
50%,4.000000
75%,4.000000
max,4.000000


In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5000 entries, 1557 to 267
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   reviews.rating  5000 non-null   int64 
 1   reviews.text    5000 non-null   object
dtypes: int64(1), object(1)
memory usage: 117.2+ KB


In [20]:
x = df['reviews.text']
y = df['reviews.rating']

In [21]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix

In [22]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df['reviews.text'], df['reviews.rating'], random_state=5)
print('Number of rows in the total set: {}'.format(df.shape[0]))
print('Number of rows in the training set: {}'.format(X_train.shape[0]))
print('Number of rows in the test set: {}'.format(X_test.shape[0]))

Number of rows in the total set: 5000
Number of rows in the training set: 3750
Number of rows in the test set: 1250


In [23]:
X_train, X_test, y_train, y_test

(2539    daughter love tablet love free time love case ...
 2203    purchas year old grandchildren use parent phon...
 3474         tablet best buy money great app great pictur
 2655                                                great
 2532                       describ would recommend friend
                               ...                        
 1277                     purchas echo plus pleas easi use
 2647    mom love kindl fire first kindl time upgrad gr...
 2219        devic good look starter tablet young individu
 2429    great tablet believ also great kid use parent ...
 1918    second kindl fire extrem pleas second one much...
 Name: reviews.text, Length: 3750, dtype: object,
 2651    learn play game learn game easi get app learn ...
 4706            complaint work great batteri awesom alway
 2570               granddaught love tablet toddler friend
 159                        echo screen add appeal product
 3075               exceed expect recommend product friend
      

In [24]:
from sklearn.feature_extraction.text import CountVectorizer
count_vector = CountVectorizer()
print(count_vector)

CountVectorizer()


In [25]:
training_data = count_vector.fit_transform(X_train)
testing_data = count_vector.transform(X_test)

In [26]:
first_element = X_test.iloc[0]
my_test_data = count_vector.transform([first_element])

In [27]:
first_element

'learn play game learn game easi get app learn download charg everi night readi get school'

In [28]:
from sklearn.naive_bayes import MultinomialNB
naive_bayes = MultinomialNB()
naive_bayes.fit(training_data, y_train)

MultinomialNB()

In [29]:
import joblib
# Assuming naive_bayes and count_vectorizer are already defined and trained
joblib.dump(naive_bayes, 'naive_bayes_model_2.joblib')

['naive_bayes_model_2.joblib']

In [30]:
joblib.dump(count_vector, 'count_vectorizer_2.joblib')

['count_vectorizer_2.joblib']

In [31]:
my_test_data_2 = 'very useful and very nice lenovo n21 Chromebook is very nice and very useful daraz shopping is very nice'

In [32]:
y_test_data_2 = count_vector.transform([my_test_data_2])
prediction_2 = naive_bayes.predict(y_test_data_2)[0]

In [33]:
prediction_2

2